In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv(r"C:\Users\balag\OneDrive\Desktop\Data Mites\Projects\Portugese Bank\PRCP-1000-ProtugeseBank\Data\bank-additional\bank-additional-full.csv")
data

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

# Exploratory Data Analysis

In [ ]:
data.corr(numeric_only = True)

In [ ]:
sns.heatmap(data.corr(numeric_only = True))

In [ ]:
data = data.drop('euribor3m',axis=1)

In [ ]:
CP = data.select_dtypes(include="object")

CP.shape

In [ ]:
HM = data.select_dtypes(include = ['int64','float64'])

HM.shape

In [ ]:
plot_number = 1
plt.figure(figsize=(10,10))
for column in HM:
    if plot_number <= 10:
        ax = plt.subplot(5,2,plot_number)
        sns.histplot(data[column],kde = True)
        plt.xlabel(column)
        plt.ylabel('Count')
    plot_number+=1
plt.tight_layout()
plt.show()
    

In [ ]:
plot_number = 1
plt.figure(figsize=(10,10))
for column in CP:
    if plot_number <= 10:
        ax = plt.subplot(5,2,plot_number)
        sns.countplot(data[column])
        plt.xlabel(column)
        plt.ylabel('Count')
    plot_number+=1
plt.tight_layout()
plt.show()
    

In [ ]:
for column in CP:
    for i in range(0,41188):
        if data[column][i] == "unknown":
            mode = data[column].mode()[0]
            data[column][i] = mode
    

In [ ]:
plot_number = 1
plt.figure(figsize=(10,10))
for column in CP:
    if plot_number <= 10:
        ax = plt.subplot(5,2,plot_number)
        sns.countplot(data[column])
        plt.xlabel(column)
        plt.ylabel('Count')
    plot_number+=1
plt.tight_layout()
plt.show()

In [ ]:
data.default.value_counts()

In [ ]:
# Chercking the Categorical Columns corretion with the Output Variable
dependency = []
No_dependency = []
for column in CP:
    contingency_table = pd.crosstab(data[column], data['y'])
    chi2_statistic, p_value, dof, expected_frequencies = chi2_contingency(contingency_table)
    alpha = 0.05
    if p_value < alpha:
        dependency.append(column)
    else:
        No_dependency.append(column)
    

In [ ]:
print(dependency)

In [ ]:
print(No_dependency)

In [ ]:
data=data.drop(['default','loan'],axis=1)

In [ ]:
le = LabelEncoder()
data['housing'] = le.fit_transform(data['housing'])
data['y'] = le.fit_transform(data['y'])

In [ ]:
to_encode = ['job','marital','education','contact','month','day_of_week','poutcome']
data_encoded = pd.get_dummies(data, columns=to_encode, dtype=int)

In [ ]:
data_encoded.columns

In [ ]:
data.columns

In [ ]:
new_order = ['age','job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed','marital_divorced', 'marital_married', 'marital_single','education_basic.4y', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree','housing','contact_cellular', 'contact_telephone','month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep','day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue', 'day_of_week_wed','duration', 'campaign', 'pdays', 'previous','poutcome_failure', 'poutcome_nonexistent', 'poutcome_success', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'nr.employed', 'y']
set(new_order) - set(data.columns)
available_cols = [c for c in new_order if c in data.columns]
processed_data = data[available_cols]

for col in new_order:
    if col not in data.columns:
        data[col] = 0

processed_data = data[new_order]


In [ ]:
print(processed_data.shape)

In [ ]:
X = processed_data.drop('y',axis=1)
y = processed_data['y']

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
X_scaled = pd.DataFrame(scaled_data, columns=X.columns)

# Seperating The independent And Dependent Columns

In [ ]:
X_scaled.head()

In [ ]:
y.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size = 0.3,random_state = 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Model Building

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
predict = lr.predict(X_test)

In [ ]:
accuracy_score(predict,y_test)

In [ ]:
print(classification_report(y_test, predict))

In [ ]:
print(confusion_matrix(y_test,predict))